In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### GGF HH production

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunesRun3ECM13p6TeV.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13600.),
                         PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        processParameters = cms.vstring(
        __CHANNEL_DECAY_FRAGMENT__
            ),
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings',
                                    'processParameters'
                                    )
        )
                         )

ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator (decay_fr):
    dict = { '2022': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2022EE': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2022","2022EE"]
processes = ["Radion", "BulkGraviton"]
mass_point = ["250", "260", "270", "280", "300", "350", "450", "550", "600", "650", "700", "800", "1000",
"1200", "1400", "1600", "1800", "2000", "2500", "3000", "4000", "5000"]
tot_events = [400000]*13+[100000]*9
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/13p6TeV/HHresonant/Spin-0'
    t_example2 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/13p6TeV/HHresonant/Spin-2'
    t_datasetname_year = 'GluGluto{process}toHHto4Tau_M-{mass}_narrow_TuneCP5_13p6TeV_madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/RunIII/13p6TeV/slc7_amd64_gcc10/MadGraph5_aMCatNLO/GF_HH_Spin{spin}/{process}_hh{graviton}narrow_M{mass}_slc7_amd64_gcc10_CMSSW_12_4_8_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    

    for process in range(len(processes)):
        #gp_ggf_year = "gp_ggf_" + years[year]
        dataset_names_year = []
        gp_ggf_year = []

        for mass in mass_point:
            if processes[process] == "Radion":
                d = {"process":processes[process], "mass": mass, "spin": "0", "graviton": "_"}
                tmp_example_dict[processes[process]] = t_example1
            elif processes[process] == "BulkGraviton":
                d = {"process":processes[process], "mass": mass, "spin": "2", "graviton": "_GF_HH_"}
                tmp_example_dict[processes[process]] = t_example2
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict[processes[process]] = dataset_names_year
            tmp_gridpack_dict[processes[process]] = gp_ggf_year
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

print(example_dict['2022']['Radion'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [ ]:
ggf_hh_tttt = '''                                                                                                                                                                                           
    '23:mMin = 0.05',
    '24:mMin = 0.05',
    '25:m0 = 125.0',
    '25:onMode = off',
    '25:onIfMatch = 15 -15',
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:mothers = 25', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 15,15,15,15',
'''

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
for process in processes:
    tmp_process_pythia_map[process] = fragmentsDictCreator(ggf_hh_tttt)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    for process in processes:
        with open('ggF_'+process+'_'+year+'.csv', 'w') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
            '''if year == "2016":
                if process == "Radion":
                    pr = 1606
                elif process == "BulkGraviton":
                    pr = 1631
            if year == "2016APV":
                if process == "Radion":
                    pr = 812
                elif process == "BulkGraviton":
                    pr = 837
            if year == "2017":
                if process == "Radion":
                    pr = 1574
                elif process == "BulkGraviton":
                    pr = 1599
            if year == "2018":
                if process == "Radion":
                    pr = 1612
                elif process == "BulkGraviton":
                    pr = 1637'''
            for i in range(len(mass_point)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.9.13"
                dataset_name = dataset_names[year][process][i]
                if year == "2022":
                    events = round(tot_events[i]*0.22)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGENAPV-00' + str(pr)
                elif year == "2022EE":
                    events = round(tot_events[i]*0.78)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                    
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process][i]) + '\n' + process_pythia_map[process][year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year][process])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Pythia8"
                mcdb_id = '0'
                time = '38'
                size = '870'
                csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1